In [7]:
import re
import json
import pickle
from string import punctuation

import nltk
import joblib
import pandas as pd
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_extraction.text import TfidfVectorizer

## Data preprocessing

In [8]:
df = pd.read_csv(
    './data/training-data/init-news.csv',
    names=['channel', 'text', 'date', 'type', 'sent']
)
df.head()

,channel,text,date,type,sent
0,раньше всех. ну почти.,президент эстонии алар карис признал удастся п...,2022-05-17 18:20:01,Economical,-1
1,раньше всех. ну почти.,россияне 24 февраля стали тратить раза новости...,2022-05-17 18:11:38,Political,-1
2,раньше всех. ну почти.,суд приговорил эксполковника захарченко совоку...,2022-05-17 17:53:12,Shelling,-1
3,раньше всех. ну почти.,евросоюз допустит украине закончилось оружие в...,2022-05-17 17:44:46,Political,-1
4,раньше всех. ну почти.,сша активно привлекают участия боевых действия...,2022-05-17 17:43:05,Political,-1


In [9]:
df['type'].value_counts()

Political       581
Shelling        377
Economical      339
Humanitarian    271
Name: type, dtype: int64

In [10]:
emoji_regex_compiled = re.compile(
    "["
    u"\U0001F600-\U0001F64F"  # emoticons
    u"\U0001F300-\U0001F5FF"  # symbols & pictographs
    u"\U0001F680-\U0001F6FF"  # transport & map symbols
    u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
    u"\U00002500-\U00002BEF"  # chinese char
    u"\U00002702-\U000027B0"
    u"\U00002702-\U000027B0"
    u"\U000024C2-\U0001F251"
    u"\U0001f926-\U0001f937"
    u"\U00010000-\U0010ffff"
    u"\u2640-\u2642"
    u"\u2600-\u2B55"
    u"\u200d"
    u"\u23cf"
    u"\u23e9"
    u"\u231a"
    u"\ufe0f"  # dingbats
    u"\u3030"
    "]+",
    re.UNICODE
)

MAX_POST_LEN_IN_WORDS = 20

with open("./config/RUSSIAN_STOP_WORDS.json") as stop_words_file:
    stop_words = json.load(stop_words_file)

In [11]:
def clean_text(text: str) -> str:
    text = re.sub(r'[0-9]', '', text)
    text = re.sub(r'<[^>]+>', ' ', text)
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = re.sub(emoji_regex_compiled, '', text)

    cleared_words = [word for word in nltk.word_tokenize(text) if word.isalpha() and word not in stop_words]
    truncated_text = cleared_words[:MAX_POST_LEN_IN_WORDS]
    text = ' '.join(truncated_text)

    text = re.sub(rf'[{punctuation}]', '', text)
    text = text.replace(' – ', ' ').replace(' - ', ' ').replace(' — ', ' ')
    text = text.replace('»', '').replace('«', '')

    text = re.sub(' +', ' ', text)

    return text

In [12]:
df = df[~df['text'].isna()]
df['text'] = df['text'].apply(clean_text)

In [13]:
df['text'].head()

0    президент эстонии алар карис признал удастся п...
1    россияне февраля стали тратить раза новости ин...
2    суд приговорил эксполковника захарченко совоку...
3    евросоюз допустит украине закончилось оружие в...
4    сша активно привлекают участия боевых действия...
Name: text, dtype: object

In [14]:
vectorizer = TfidfVectorizer()

texts = df['text']
train_vectors = vectorizer.fit_transform(texts)

In [15]:
pickle.dump(vectorizer, open('./trained-models/vectorizer.pk', 'wb'))

In [16]:
types = pd.factorize(df['type'])
types[1]

Index(['Economical', 'Political', 'Shelling', 'Humanitarian'], dtype='object')

In [17]:
df['type'] = types[0]

In [18]:
df['type'].head(3)

0    0
1    1
2    2
Name: type, dtype: int64

## SVC Model

In [19]:
svc = SVC(kernel='linear')

svc = svc.fit(train_vectors, df['type'])

In [20]:
joblib.dump(svc, open('./trained-models/svc-news-type-prediction.sav', 'wb'))

# Gaussian

In [21]:
nb = GaussianNB()

nb = nb.fit(train_vectors.toarray(), df['type'])

In [22]:
joblib.dump(nb, open('./trained-models/nb-news-type-prediction.sav', 'wb'))

## KNN Model

In [23]:
knn = KNeighborsClassifier(n_neighbors=8)
knn = knn.fit(train_vectors, df['type'])

In [24]:
joblib.dump(knn, open('./trained-models/knn-news-type-prediction.sav', 'wb'))